In [3]:
import pandas as pd
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

### 전처리 함수
#### path : 데이터를 불러올 경로 설정
#### slice : 가져올 열 이름
#### column_ : 가져온 데이터의 열 이름 변경. 날짜 데이터의 이름은 Date로 고정. 이유는 merge기준을 Date로 설정했기 때문
#### encoding : 데이터를 불러올 때 필요한 인코딩 형식 설정
#### header : 기본적으로 있는 header를 가져오지만 header가 없는 경우 첫 행의 데이터가 header로 지정될 수 있기 떄문에 header가 없을 경우는 None으로 해줘야 함

In [5]:
def preprocess_(path,  slice, column_, encoding=None, header='infer'):

    df = pd.read_csv(path, encoding=encoding, header=header)
    
    if slice:
        df = df[slice]

    df.columns = column_

    if (type(df.iloc[:,1][1])==str):
        df.iloc[:,1] = df.iloc[:,1].apply(lambda x: float(x.replace(',', '')))
        
    df['Date'] = df['Date'].str.replace('[^0-9]', '')
    df['Date'] = df['Date'].apply(lambda x: x[:4]+'-'+x[4:6]+'-'+x[6:])
    df['Date'] = pd.to_datetime(df['Date'])

    df.sort_values('Date', inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index'], axis=1, inplace=True)

    return df

#### before data 전처리 및 병합

In [6]:
before_ETF = preprocess_(path='./datasets/before_ETF.csv', slice=['Date', "Close"], column_=['Date', 'ETF'], encoding='CP949')
before_WTI = preprocess_(path='./datasets/before_WTI.csv', slice=['날짜', "종가"], column_=['Date', 'WTI'])
before_ER = preprocess_(path='./datasets/before_ExchangeRate.csv', slice=[0,1], header=None, column_=['Date', 'ExchangeRate'], encoding='CP949')
before_gold = preprocess_(path='./datasets/before_gold.csv', slice=['일자', "종가"], column_=['Date', 'Gold'], encoding='CP949')
before_pork =preprocess_(path='./datasets/before_pork.csv', slice=['구분', "평균"], column_=['Date', 'Pork'])
before_ITA = preprocess_(path='./datasets/before_ITA.csv', slice=['Date', "ITA_Close"], column_=['Date', 'ITA'], encoding='CP949')

from functools import reduce
dfs = [before_ETF, before_WTI, before_ER, before_gold, before_pork, before_ITA]
union = reduce(lambda left, right: pd.merge(left, right, on='Date', how='inner'),dfs)
union
#union.to_csv('before_data.csv', index=None)

,Date,ETF,WTI,ExchangeRate,Gold,Pork,ITA
0,2020-11-02,8045,36.81,1135.5,68650,2062.0,77.44
1,2020-11-03,8120,37.66,1133.0,69010,2061.0,79.91
2,2020-11-04,8165,39.15,1140.0,69400,2039.0,79.87
3,2020-11-05,8335,38.79,1127.5,69250,2186.0,80.80
4,2020-11-06,8355,37.14,1122.5,69930,2181.0,80.73
...,...,...,...,...,...,...,...
93,2021-03-25,10045,58.56,1136.1,63220,2124.0,102.28
94,2021-03-26,9985,60.97,1131.5,62770,2124.0,102.64
95,2021-03-29,9950,61.56,1133.5,62840,2151.0,103.18
96,2021-03-30,9880,60.55,1134.5,62110,2157.0,103.84


#### after data 전처리 및 병합

In [4]:
after_ETF = preprocess_(path='./datasets/after_ETF.csv', slice=['Date', "Close"], column_=['Date', 'ETF'], encoding='CP949')
after_WTI = preprocess_(path='./datasets/after_WTI.csv', slice=['날짜', "종가"], column_=['Date', 'WTI'])
after_ER = preprocess_(path='./datasets/after_ExchangeRate.csv', slice=[0,1], header=None, column_=['Date', 'ExchangeRate'], encoding='CP949')
after_gold = preprocess_(path='./datasets/after_gold.csv', slice=['일자', "종가"], column_=['Date', 'Gold'], encoding='CP949')
after_pork =preprocess_(path='./datasets/after_pork.csv', slice=['구분', "평균"], column_=['Date', 'Pork'], encoding='CP949')
after_ITA = preprocess_(path='./datasets/after_ITA.csv', slice=['Date', "ITA_Close"], column_=['Date', 'ITA'], encoding='CP949')

from functools import reduce
dfs = [after_ETF, after_WTI, after_ER, after_gold, after_pork, after_ITA]
union = reduce(lambda left, right: pd.merge(left, right, on='Date', how='inner'),dfs)
union
union.to_csv('after_data.csv', index=None)